## Top Performing genres and companies

In [1]:
!pip install psycopg2-binary

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 2.1 MB/s eta 0:00:0000:0100:01


In [2]:
import psycopg2
import pandas as pd

class PostgresClient:
    def __init__(self):
        """Initializes the PostgresClient with connection parameters."""
        self.host = 'postgres'
        self.port = 5432
        self.user = 'myuser'
        self.password = 'mypassword'
        self.database = 'mydatabase'

    def execute_query(self, query):
        """Executes a SELECT statement and returns the results as a DataFrame.

        Args:
            query: The SELECT statement to execute.

        Returns:
            A DataFrame containing the results of the SELECT statement.
        """

        # Connect to the database
        conn = psycopg2.connect(
            host=self.host,
            port=self.port,
            user=self.user,
            password=self.password,
            dbname=self.database
        )

        # Create a cursor
        cur = conn.cursor()

        # Execute the SELECT statement
        cur.execute(query)
        
        # Get the column names
        column_names = [column[0] for column in cur.description]
        

        # Fetch the results of the SELECT statement
        rows = cur.fetchall()

        # Convert the rows to a DataFrame
        df = pd.DataFrame(rows, columns=column_names)

        # Close the cursor
        cur.close()

        # Close the connection
        conn.close()

        return df
    
    def get_table_schema(self, table_name):
        """Gets the schema of a table in the database.

        Args:
            table_name: The name of the table.

        Returns:
            A DataFrame containing the column names and data types of the table.
        """
        
        query = f"""
                SELECT column_name, data_type
                FROM information_schema.columns
                WHERE table_name = '{table_name}'
                """

        schema_df = self.execute_query(query)
        
        return schema_df
    
    def explode_table(self, table_name, explode_column):
        """Explodes an array column in a table and returns the resulting table.

        Args:
            table_name: The name of the table.
            explode_column: The name of the array column to explode.

        Returns:
            A DataFrame containing the resulting table after exploding the array column.
        """

        # Select all columns and explode the array column
        query = f"""
            SELECT *, unnest("{explode_column}") as "{explode_column}_exploded"
            FROM "{table_name}"
            """

        # Execute the query and return the resulting table
        df = self.execute_query(query)

        # Drop the original array column
        df = df.drop(columns=[explode_column])
        
        return df
    
    def select_top_performing(self, table_name, explode_column, order_by, n):
        """Explodes an array column in a table, orders the resulting table by a given column, and returns the top n rows.

        Args:
            table_name: The name of the table.
            explode_column: The name of the array column to explode.
            order_by: The column to order the resulting table by.
            n: The number of top rows to return.

        Returns:
            A DataFrame containing the top n rows of the resulting table after exploding the array column and ordering by the given column.
        """

        # Explode the array columns in the table
        df = self.explode_table(table_name, explode_column)

        # Order the resulting table by the given column and return the top n rows
        df = df.sort_values(by=order_by, ascending=False).head(n)
        
        return df


        


In [3]:
client = PostgresClient()

In [4]:
client.execute_query("SELECT * FROM movies;")

,title,year,production_companies,genres,rating,revenue,budget,revenue_to_budget,url,abstract
0,King Kong,1933,[RKO Radio Pictures],"[Action, Adventure, Drama, Horror]",7.7,10000000,672000,14.88,https://en.wikipedia.org/wiki/King_Kong_(1933_...,| producer =
1,King Kong,1976,"[Paramount Pictures, Dino De Laurentiis Company]","[Adventure, Drama]",5.9,90614445,23000000,3.94,https://en.wikipedia.org/wiki/King_Kong_(1976_...,| starring = Jeff BridgesCharles GrodinJessica...
2,King Kong,2005,"[WingNut Films, Universal Pictures, Big Primat...","[Adventure, Drama, Action]",6.6,550000000,207000000,2.66,None,None
3,King Kong Lives,1986,[De Laurentiis Entertainment Group (DEG)],"[Action, Horror]",4.6,4711220,10000000,0.47,None,None
4,Antwone Fisher,2002,"[Fox Searchlight Pictures, Mundy Lane Entertai...","[Drama, Romance]",6.7,23367586,12500000,1.87,https://en.wikipedia.org/wiki/Antwone_Fisher_(...,| producer =
5,Mozart and the Whale,2005,[],"[Comedy, Drama, Romance]",6.5,84967,12000000,0.01,None,None


In [5]:
client.get_table_schema(table_name="movies")

,column_name,data_type
0,revenue_to_budget,double precision
1,revenue,bigint
2,budget,bigint
3,genres,ARRAY
4,rating,text
5,url,text
6,title,text
7,abstract,text
8,year,text
9,production_companies,ARRAY


In [6]:
exploded_df = client.explode_table('movies', 'production_companies')
exploded_df

,title,year,genres,rating,revenue,budget,revenue_to_budget,url,abstract,production_companies_exploded
0,King Kong,1933,"[Action, Adventure, Drama, Horror]",7.7,10000000,672000,14.88,https://en.wikipedia.org/wiki/King_Kong_(1933_...,| producer =,RKO Radio Pictures
1,King Kong,1976,"[Adventure, Drama]",5.9,90614445,23000000,3.94,https://en.wikipedia.org/wiki/King_Kong_(1976_...,| starring = Jeff BridgesCharles GrodinJessica...,Paramount Pictures
2,King Kong,1976,"[Adventure, Drama]",5.9,90614445,23000000,3.94,https://en.wikipedia.org/wiki/King_Kong_(1976_...,| starring = Jeff BridgesCharles GrodinJessica...,Dino De Laurentiis Company
3,King Kong,2005,"[Adventure, Drama, Action]",6.6,550000000,207000000,2.66,None,None,WingNut Films
4,King Kong,2005,"[Adventure, Drama, Action]",6.6,550000000,207000000,2.66,None,None,Universal Pictures
5,King Kong,2005,"[Adventure, Drama, Action]",6.6,550000000,207000000,2.66,None,None,Big Primate Pictures
6,King Kong,2005,"[Adventure, Drama, Action]",6.6,550000000,207000000,2.66,None,None,MFPV Film
7,King Kong Lives,1986,"[Action, Horror]",4.6,4711220,10000000,0.47,None,None,De Laurentiis Entertainment Group (DEG)
8,Antwone Fisher,2002,"[Drama, Romance]",6.7,23367586,12500000,1.87,https://en.wikipedia.org/wiki/Antwone_Fisher_(...,| producer =,Fox Searchlight Pictures
9,Antwone Fisher,2002,"[Drama, Romance]",6.7,23367586,12500000,1.87,https://en.wikipedia.org/wiki/Antwone_Fisher_(...,| producer =,Mundy Lane Entertainment


In [7]:
top_performing_generes = client.select_top_performing(table_name='movies', explode_column='production_companies', order_by='revenue_to_budget', n=3)
top_performing_generes

,title,year,genres,rating,revenue,budget,revenue_to_budget,url,abstract,production_companies_exploded
0,King Kong,1933,"[Action, Adventure, Drama, Horror]",7.7,10000000,672000,14.88,https://en.wikipedia.org/wiki/King_Kong_(1933_...,| producer =,RKO Radio Pictures
1,King Kong,1976,"[Adventure, Drama]",5.9,90614445,23000000,3.94,https://en.wikipedia.org/wiki/King_Kong_(1976_...,| starring = Jeff BridgesCharles GrodinJessica...,Paramount Pictures
2,King Kong,1976,"[Adventure, Drama]",5.9,90614445,23000000,3.94,https://en.wikipedia.org/wiki/King_Kong_(1976_...,| starring = Jeff BridgesCharles GrodinJessica...,Dino De Laurentiis Company


In [8]:
top_performing_generes['production_companies_exploded'].unique()

array(['RKO Radio Pictures', 'Paramount Pictures',
       'Dino De Laurentiis Company'], dtype=object)